# Hyperdimensional Computing — `kongming-rs-hv`

An introduction to sparse binary hypervectors, operators, and composite structures
using the Rust-backed `kongming-rs-hv` package.


In [ ]:
# Install / upgrade kongming-rs-hv (required for Google Colab).
# After this cell finishes, restart the runtime before running any other cell:
#   Runtime menu → Restart session  (or Ctrl+M .)
import subprocess, sys
result = subprocess.run(
    [sys.executable, '-m', 'pip', 'install', '--upgrade', 'kongming-rs-hv'],
    capture_output=True, text=True
)
print(result.stdout[-800:] if result.stdout else '')
if result.returncode != 0:
    print('ERROR:', result.stderr[-800:])
print("Restart the kernel before importing (Kernel → Restart Kernel).")

In [ ]:
import importlib.metadata
importlib.metadata.version("kongming-rs-hv")

## Imports

In [ ]:
from kongming_rs.hv import (
    # model constants
    MODEL_64K_8BIT, MODEL_1M_10BIT, MODEL_16M_12BIT,
    # prewired constants
    PREWIRED_STEP, PREWIRED_SET_MARKER, PREWIRED_SEQUENCE_MARKER, PREWIRED_OCTOPUS_MARKER,
    # types
    Domain, Pod, Seed128, Exp,
    SparseOperation, SparseSegmented,
    Sparkle, Knot, Parcel, Set, Sequence, Octopus, Cyclone, Learner, Replacement,
    # shortcuts
    d0, p0,
    # operators
    bind, bundle, release, replace, replace_single, inverse,
    # similarity
    overlap, hamming, jaccard_index, equal, is_identity,
    # proto
    to_message, from_message,
)

## `SparseOperation`

`SparseOperation` models sparse operations for stochastic hypervectors and provides the
pseudo-random number generator used across various operations.

Create one directly — the second and third arguments are the initial 128-bit seeds
(two uint64 values) for the internal RNG.

In [ ]:
so = SparseOperation(MODEL_1M_10BIT, 0, 99)
so.model(), so.width()

### Generating hypervectors

In [ ]:
a = Sparkle.random(d0(), so)
b = Sparkle.random(d0(), so)
hex(a.stable_hash())

Two random vectors are nearly orthogonal:

In [ ]:
overlap(a, b), hamming(a, b)

Vectors seeded by an integer or a word:

In [ ]:
c = Sparkle.from_seed(MODEL_1M_10BIT, d0(), 1234)
d = Sparkle.from_word(MODEL_1M_10BIT, d0(), "random")
e = Sparkle.from_word(MODEL_1M_10BIT, d0(), "RANDOM")
str(c), str(d), str(e)

In [ ]:
equal(d, e), overlap(a, c), overlap(a, d)

### Identity vector

In [ ]:
identity = SparseSegmented.identity(so.model())
str(identity), is_identity(identity)

### Inspection via `core()` and `proto_bytes()`

In [ ]:
core_a = a.core()
type(core_a), core_a.offsets()[:8]   # first 8 offsets

In [ ]:
# stable_hash is representation-agnostic
a.stable_hash() == core_a.stable_hash()

Proto serialization round-trip:

In [ ]:
msg = to_message(a)
msg

In [ ]:
back = from_message(msg)
equal(back, a), back.stable_hash() == a.stable_hash()

### The `bundle` operator

`Seed128(high, low)` constructs a 128-bit bundle seed from two uint64 values.

In [ ]:
bundled0 = bundle(Seed128(0, 0), a, c)
overlap(bundled0, a), overlap(bundled0, c)

In [ ]:
bundled1 = bundle(Seed128(0, 1), a, c)
overlap(bundled1, a), overlap(bundled1, c), equal(bundled0, bundled1)

### The `bind` operator

In [ ]:
bound = bind(a, c)
type(bound), overlap(bound, a), overlap(bound, c)

### Domains and Pods

`Domain` and `Pod` act as logical namespaces and slots within those namespaces.

In [ ]:
d1 = Domain.from_name("my_domain")
d2 = Domain.from_id(2)
str(d1), str(d2)

In [ ]:
p_seed    = Pod.from_seed(1234)
p_step    = Pod.from_prewired(PREWIRED_STEP)
p_word    = Pod.from_word("first_pod")
str(p_seed), str(p_step), str(p_word)

In [ ]:
step = Sparkle.from_prewired(so.model(), d1, PREWIRED_STEP)
str(a), str(d), str(step)

## Composites

### Sets

$S = S_{marker} \otimes (\sum_{i,\oplus} M_i)$

In [ ]:
set0 = Set(d1, Pod.from_seed(1234), a, b, c, d)
to_message(set0)

In [ ]:
set0_marker  = Sparkle.from_prewired(so.model(), d1, PREWIRED_SET_MARKER)
set0_stripped = bind(set0, inverse(set0_marker))

(overlap(set0_stripped, a), overlap(set0_stripped, b),
 overlap(set0_stripped, c), overlap(set0_stripped, d))

### Sequences

$S = S_{marker} \otimes (\sum_{i,\oplus} M_i \otimes S_{step}^{i})$

In [ ]:
seq0 = Sequence(d1, Pod.from_seed(1234), a, b, c, d)
to_message(seq0)

In [ ]:
seq0_marker  = Sparkle.from_prewired(so.model(), d1, PREWIRED_SEQUENCE_MARKER)
seq0_stripped = bind(seq0, inverse(seq0_marker))
step          = Sparkle.from_prewired(so.model(), d1, PREWIRED_STEP)

(overlap(seq0_stripped, a),
 overlap(bind(seq0_stripped, step.power(-1)), b),
 overlap(bind(seq0_stripped, step.power(-2)), c),
 overlap(bind(seq0_stripped, step.power(-3)), d))

### Key-value pairs (Octopus)

$S = S_{marker} \otimes (\sum_{i,\oplus} K_i \otimes M_i)$


In [ ]:
octopus0 = Octopus(d1, Pod.from_seed(1234), ["first", "second", "third", "fourth"], a, b, c, d)
to_message(octopus0)

In [ ]:
octopus0_marker  = Sparkle.from_prewired(so.model(), d1, PREWIRED_OCTOPUS_MARKER)
octopus0_stripped = bind(octopus0, inverse(octopus0_marker))

key0 = Sparkle.from_word(so.model(), d1, "first")
key1 = Sparkle.from_word(so.model(), d1, "second")
key2 = Sparkle.from_word(so.model(), d1, "third")
key3 = Sparkle.from_word(so.model(), d1, "fourth")

(overlap(octopus0_stripped, bind(key0, a)),
 overlap(octopus0_stripped, bind(key1, b)),
 overlap(octopus0_stripped, bind(key2, c)),
 overlap(octopus0_stripped, bind(key3, d)))

## Learners

Learners are mutable hypervectors that accumulate observations over time.

In [ ]:
learner0 = Learner(so.model(), d0(), Pod.from_seed(1234))
learner1 = Learner.random(so)
str(learner0)

In [ ]:
learner0.bundle(a)
str(learner0)

In [ ]:
learner0.weight(a)

In [ ]:
learner0.bundle(b)
learner0.bundle(c)
learner0.bundle(d)

learner0.weight(a), overlap(learner0, a)

In [ ]:
learner0.bundle(a)
learner0.weight(a)   # a appears 2 out of 5 observations

In [ ]:
learner0.bundle_multiple(b, 3)
learner0.weight(b)   # b appears 4 out of 8 observations